<h1>Chapter 2</h1> 

1. Spark run on a number of separate machines __Cluster__
2. Spark application is comprised of a __driver__ process and a set of __executors__
3. The __driver__ process runs the ***main()*** function 
4. The driver is responsible for:
	- Maintaining information about the Spark application
	- Responding to user's program or input
	- Analyzing, distributing, and scheduling work across executors
5. Each executor is responsible ofr the following:
	- Execute code assigned by the __driver__
	- Reporting the state of the computation on the executor back to the __driver__ node

<p align="center">
	<img src="https://miro.medium.com/max/1058/1*jLVo8Bl4m9pXhADFxzbDVA.png"/>
</p>

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
	.config("sspark.executor..instances","2")
	.config("spark.executor.memory","2G")
    .getOrCreate()
}

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/09/12 15:03:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@293befa0

![Spark Eco-System](https://miro.medium.com/max/1400/1*fQ5VWad1nu4yYt2MOuSBsQ.png)

In [4]:
val myRange = spark.range(100).toDF("number")
myRange.show(5)

21/09/12 15:00:15 DEBUG FileSystem: Loading filesystems
21/09/12 15:00:15 DEBUG FileSystem: file:// = class org.apache.hadoop.fs.LocalFileSystem from /Users/mehdi/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.2.0/hadoop-common-3.2.0.jar
21/09/12 15:00:15 DEBUG FileSystem: viewfs:// = class org.apache.hadoop.fs.viewfs.ViewFileSystem from /Users/mehdi/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.2.0/hadoop-common-3.2.0.jar
21/09/12 15:00:15 DEBUG FileSystem: har:// = class org.apache.hadoop.fs.HarFileSystem from /Users/mehdi/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.2.0/hadoop-common-3.2.0.jar
21/09/12 15:00:15 DEBUG FileSystem: http:// = class org.apache.hadoop.fs.http.HttpFileSystem from /Users/mehdi/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.2.0/hadoop-common-3.2.0.jar
21/09/12 15:00:15 DEBUG FileSystem:

show at cmd3.sc:2

1 / 1

21/09/12 15:00:18 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 9.9 KiB, free 2004.6 MiB)
21/09/12 15:00:18 DEBUG BlockManager: Put block broadcast_0 locally took 35 ms
21/09/12 15:00:18 DEBUG BlockManager: Putting block broadcast_0 without replication took 37 ms
21/09/12 15:00:18 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 4.6 KiB, free 2004.6 MiB)
21/09/12 15:00:18 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_0_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:18 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 192.168.0.71:49690 (size: 4.6 KiB, free: 2004.6 MiB)
21/09/12 15:00:18 DEBUG BlockManagerMaster: Updated info of block broadcast_0_piece0
21/09/12 15:00:18 DEBUG BlockManager: Told master about block broadcast_0_piece0
21/09/12 15:00:18 DEBUG BlockManager: Put block broadcast_0_piece0 locally took 10 ms
21/09/12 15:00:18 DEBUG BlockManage

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows



myRange: DataFrame = [number: bigint]

In [5]:
val divisBy2 = myRange.where("number % 2 = 0")
divisBy2.show(5)

21/09/12 15:00:31 DEBUG TorrentBroadcast: Unpersisting TorrentBroadcast 0
21/09/12 15:00:31 DEBUG BlockManagerStorageEndpoint: removing broadcast 0
21/09/12 15:00:31 DEBUG BlockManager: Removing broadcast 0
21/09/12 15:00:31 DEBUG BlockManager: Removing block broadcast_0_piece0
21/09/12 15:00:31 DEBUG MemoryStore: Block broadcast_0_piece0 of size 4738 dropped from memory (free 2101965329)
21/09/12 15:00:31 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_0_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:31 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 192.168.0.71:49690 in memory (size: 4.6 KiB, free: 2004.6 MiB)
21/09/12 15:00:31 DEBUG BlockManagerMaster: Updated info of block broadcast_0_piece0
21/09/12 15:00:31 DEBUG BlockManager: Told master about block broadcast_0_piece0
21/09/12 15:00:31 DEBUG BlockManager: Removing block broadcast_0
21/09/12 15:00:31 DEBUG MemoryStore: Block broadcast_0 of size 10120 dropped from memory

show at cmd4.sc:2

1 / 1

21/09/12 15:00:32 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 10.6 KiB, free 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManager: Put block broadcast_1 locally took 1 ms
21/09/12 15:00:32 DEBUG BlockManager: Putting block broadcast_1 without replication took 1 ms
21/09/12 15:00:32 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 4.9 KiB, free 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_1_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:32 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on 192.168.0.71:49690 (size: 4.9 KiB, free: 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManagerMaster: Updated info of block broadcast_1_piece0
21/09/12 15:00:32 DEBUG BlockManager: Told master about block broadcast_1_piece0
21/09/12 15:00:32 DEBUG BlockManager: Put block broadcast_1_piece0 locally took 2 ms
21/09/12 15:00:32 DEBUG BlockManager:

show at cmd4.sc:2

1 / 1

21/09/12 15:00:32 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 10.6 KiB, free 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManager: Put block broadcast_2 locally took 1 ms
21/09/12 15:00:32 DEBUG BlockManager: Putting block broadcast_2 without replication took 1 ms
21/09/12 15:00:32 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 4.9 KiB, free 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_2_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:32 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.0.71:49690 (size: 4.9 KiB, free: 2004.6 MiB)
21/09/12 15:00:32 DEBUG BlockManagerMaster: Updated info of block broadcast_2_piece0
21/09/12 15:00:32 DEBUG BlockManager: Told master about block broadcast_2_piece0
21/09/12 15:00:32 DEBUG BlockManager: Put block broadcast_2_piece0 locally took 2 ms
21/09/12 15:00:32 DEBUG BlockManager:

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
+------+
only showing top 5 rows



divisBy2: Dataset[Row] = [number: bigint]

In [6]:
divisBy2.count()

21/09/12 15:00:52 DEBUG WholeStageCodegenExec: 
/* 001 */ public Object generate(Object[] references) {
/* 002 */   return new GeneratedIteratorForCodegenStage2(references);
/* 003 */ }
/* 004 */
/* 005 */ // codegenStageId=2
/* 006 */ final class GeneratedIteratorForCodegenStage2 extends org.apache.spark.sql.execution.BufferedRowIterator {
/* 007 */   private Object[] references;
/* 008 */   private scala.collection.Iterator[] inputs;
/* 009 */   private boolean agg_initAgg_0;
/* 010 */   private boolean agg_bufIsNull_0;
/* 011 */   private long agg_bufValue_0;
/* 012 */   private scala.collection.Iterator inputadapter_input_0;
/* 013 */   private org.apache.spark.sql.catalyst.expressions.codegen.UnsafeRowWriter[] agg_mutableStateArray_0 = new org.apache.spark.sql.catalyst.expressions.codegen.UnsafeRowWriter[1];
/* 014 */
/* 015 */   public GeneratedIteratorForCodegenStage2(Object[] references) {
/* 016 */     this.references = references;
/* 017 */   }
/* 018 */
/* 019 */   public vo

count at cmd5.sc:1 
 (kill)

0 / 12

21/09/12 15:00:52 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 14.2 KiB, free 2004.6 MiB)
21/09/12 15:00:52 DEBUG BlockManager: Put block broadcast_3 locally took 1 ms
21/09/12 15:00:52 DEBUG BlockManager: Putting block broadcast_3 without replication took 1 ms
21/09/12 15:00:52 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 6.8 KiB, free 2004.5 MiB)
21/09/12 15:00:52 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_3_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:52 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.0.71:49690 (size: 6.8 KiB, free: 2004.6 MiB)
21/09/12 15:00:52 DEBUG BlockManagerMaster: Updated info of block broadcast_3_piece0
21/09/12 15:00:52 DEBUG BlockManager: Told master about block broadcast_3_piece0
21/09/12 15:00:52 DEBUG BlockManager: Put block broadcast_3_piece0 locally took 2 ms
21/09/12 15:00:52 DEBUG BlockManager:

count at cmd5.sc:1 
 (kill)

0 / 1

21/09/12 15:00:52 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 10.1 KiB, free 2004.6 MiB)
21/09/12 15:00:52 DEBUG BlockManager: Put block broadcast_4 locally took 1 ms
21/09/12 15:00:52 DEBUG BlockManager: Putting block broadcast_4 without replication took 1 ms
21/09/12 15:00:52 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 5.0 KiB, free 2004.6 MiB)
21/09/12 15:00:52 DEBUG BlockManagerMasterEndpoint: Updating block info on master broadcast_4_piece0 for BlockManagerId(driver, 192.168.0.71, 49690, None)
21/09/12 15:00:52 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 192.168.0.71:49690 (size: 5.0 KiB, free: 2004.6 MiB)
21/09/12 15:00:52 DEBUG BlockManagerMaster: Updated info of block broadcast_4_piece0
21/09/12 15:00:52 DEBUG BlockManager: Told master about block broadcast_4_piece0
21/09/12 15:00:52 DEBUG BlockManager: Put block broadcast_4_piece0 locally took 1 ms
21/09/12 15:00:52 DEBUG BlockManager:

In [5]:
val flightData2015 = spark.read
						  .option("inferSchema","true")
						  .option("header","true")
						  .csv("/Users/mehdi/Documents/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

21/08/13 20:04:02 INFO InMemoryFileIndex: It took 13 ms to list leaf files for 1 paths.
21/08/13 20:04:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
21/08/13 20:04:03 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 20:04:03 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#20, None)) > 0)
21/08/13 20:04:03 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
21/08/13 20:04:03 INFO CodeGenerator: Code generated in 10.480572 ms
21/08/13 20:04:03 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 391.3 KiB, free 2004.2 MiB)
21/08/13 20:04:03 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 37.8 KiB, free 2004.1 MiB)
21/08/13 20:04:03 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on 192.168.0.68:57385 (size: 37.8 KiB, free: 2004.6 MiB)
21/08/13 20:04:03 INFO SparkContext: Created broadcast 5 from csv at cmd4.sc:4
21/08/13 20:04:03 INFO FileSourceScanExec: Plan

csv at cmd4.sc:4

1 / 1

21/08/13 20:04:03 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 10.8 KiB, free 2004.1 MiB)
21/08/13 20:04:03 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 5.4 KiB, free 2004.1 MiB)
21/08/13 20:04:03 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 192.168.0.68:57385 (size: 5.4 KiB, free: 2004.5 MiB)
21/08/13 20:04:03 INFO SparkContext: Created broadcast 6 from broadcast at DAGScheduler.scala:1383
21/08/13 20:04:03 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 5 (MapPartitionsRDD[18] at csv at cmd4.sc:4) (first 15 tasks are for partitions Vector(0))
21/08/13 20:04:03 INFO TaskSchedulerImpl: Adding task set 5.0 with 1 tasks resource profile 0
21/08/13 20:04:03 INFO TaskSetManager: Starting task 0.0 in stage 5.0 (TID 16) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4919 bytes) taskResourceAssignments Map()
21/08/13 20:04:03 INFO Executor: Running task 0.0 in stage 5.0 (TID 1

csv at cmd4.sc:4

1 / 1

21/08/13 20:04:03 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 59.3 KiB, free 2003.7 MiB)
21/08/13 20:04:03 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 17.5 KiB, free 2003.6 MiB)
21/08/13 20:04:03 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on 192.168.0.68:57385 (size: 17.5 KiB, free: 2004.5 MiB)
21/08/13 20:04:03 INFO SparkContext: Created broadcast 8 from broadcast at DAGScheduler.scala:1383
21/08/13 20:04:03 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 6 (MapPartitionsRDD[24] at csv at cmd4.sc:4) (first 15 tasks are for partitions Vector(0))
21/08/13 20:04:03 INFO TaskSchedulerImpl: Adding task set 6.0 with 1 tasks resource profile 0
21/08/13 20:04:03 INFO TaskSetManager: Starting task 0.0 in stage 6.0 (TID 17) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4919 bytes) taskResourceAssignments Map()
21/08/13 20:04:03 INFO Executor: Running task 0.0 in stage 6.0 (TID

flightData2015: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [6]:
flightData2015.take(3)

21/08/13 20:04:19 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 20:04:19 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 20:04:19 INFO FileSourceStrategy: Output Data Schema: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int ... 1 more fields>
21/08/13 20:04:19 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 391.2 KiB, free 2003.3 MiB)
21/08/13 20:04:19 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 37.8 KiB, free 2003.2 MiB)
21/08/13 20:04:19 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on 192.168.0.68:57385 (size: 37.8 KiB, free: 2004.5 MiB)
21/08/13 20:04:19 INFO SparkContext: Created broadcast 9 from take at cmd5.sc:1
21/08/13 20:04:19 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
21/08/13 20:04:19 INFO SparkContext: Starting job: take at cmd5.sc:1
21/08/13 20:04:19 INFO DAGSchedul

take at cmd5.sc:1

1 / 1

21/08/13 20:04:19 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 9.3 KiB, free 2003.2 MiB)
21/08/13 20:04:19 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 5.2 KiB, free 2003.2 MiB)
21/08/13 20:04:19 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on 192.168.0.68:57385 (size: 5.2 KiB, free: 2004.5 MiB)
21/08/13 20:04:19 INFO SparkContext: Created broadcast 10 from broadcast at DAGScheduler.scala:1383
21/08/13 20:04:19 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 7 (MapPartitionsRDD[27] at take at cmd5.sc:1) (first 15 tasks are for partitions Vector(0))
21/08/13 20:04:19 INFO TaskSchedulerImpl: Adding task set 7.0 with 1 tasks resource profile 0
21/08/13 20:04:19 INFO TaskSetManager: Starting task 0.0 in stage 7.0 (TID 18) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4919 bytes) taskResourceAssignments Map()
21/08/13 20:04:19 INFO Executor: Running task 0.0 in stage 7.0 (T

res5: Array[Row] = Array(
  [United States,Romania,15],
  [United States,Croatia,1],
  [United States,Ireland,344]
)

In [7]:

flightData2015.createOrReplaceTempView("flight_data_2015")

In [8]:
val sqlWay = spark.sql("""
  SELECT DEST_COUNTRY_NAME, count(1)
  FROM flight_data_2015
  GROUP BY DEST_COUNTRY_NAME
  """)

sqlWay: DataFrame = [DEST_COUNTRY_NAME: string, count(1): bigint]

In [8]:
val dataFrameWay = flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .count()

dataFrameWay: DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]

In [10]:
sqlWay.explain

21/08/13 19:54:22 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 19:54:22 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 19:54:22 INFO FileSourceStrategy: Output Data Schema: struct<DEST_COUNTRY_NAME: string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#36], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#36, 200), ENSURE_REQUIREMENTS, [id=#96]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#36], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#36] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/mehdi/Documents/Spark-The-Definitive-Guide/data/flight-data/csv/201..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [11]:
dataFrameWay.explain

21/08/13 19:54:24 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 19:54:24 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 19:54:24 INFO FileSourceStrategy: Output Data Schema: struct<DEST_COUNTRY_NAME: string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#36], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#36, 200), ENSURE_REQUIREMENTS, [id=#115]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#36], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#36] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/mehdi/Documents/Spark-The-Definitive-Guide/data/flight-data/csv/201..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [9]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

21/08/13 20:04:56 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 20:04:56 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 20:04:56 INFO FileSourceStrategy: Output Data Schema: struct<count: int>
21/08/13 20:04:56 INFO CodeGenerator: Code generated in 11.773479 ms
21/08/13 20:04:56 INFO CodeGenerator: Code generated in 12.66502 ms
21/08/13 20:04:56 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 391.2 KiB, free 2002.8 MiB)
21/08/13 20:04:56 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 37.8 KiB, free 2002.8 MiB)
21/08/13 20:04:56 INFO BlockManagerInfo: Added broadcast_11_piece0 in memory on 192.168.0.68:57385 (size: 37.8 KiB, free: 2004.4 MiB)
21/08/13 20:04:56 INFO SparkContext: Created broadcast 11 from take at cmd8.sc:1
21/08/13 20:04:56 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
21/08/13 20:04:56 INFO SparkContext

take at cmd8.sc:1

1 / 1

21/08/13 20:04:56 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 15.8 KiB, free 2002.8 MiB)
21/08/13 20:04:56 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 8.0 KiB, free 2002.8 MiB)
21/08/13 20:04:56 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on 192.168.0.68:57385 (size: 8.0 KiB, free: 2004.4 MiB)
21/08/13 20:04:56 INFO SparkContext: Created broadcast 12 from broadcast at DAGScheduler.scala:1383
21/08/13 20:04:56 INFO DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 8 (MapPartitionsRDD[31] at take at cmd8.sc:1) (first 15 tasks are for partitions Vector(0))
21/08/13 20:04:56 INFO TaskSchedulerImpl: Adding task set 8.0 with 1 tasks resource profile 0
21/08/13 20:04:56 INFO TaskSetManager: Starting task 0.0 in stage 8.0 (TID 19) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4908 bytes) taskResourceAssignments Map()
21/08/13 20:04:56 INFO Executor: Running task 0.0 in stage 8

take at cmd8.sc:1

1 / 1

21/08/13 20:04:56 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 10.6 KiB, free 2002.8 MiB)
21/08/13 20:04:56 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 5.3 KiB, free 2002.7 MiB)
21/08/13 20:04:56 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on 192.168.0.68:57385 (size: 5.3 KiB, free: 2004.4 MiB)
21/08/13 20:04:56 INFO SparkContext: Created broadcast 13 from broadcast at DAGScheduler.scala:1383
21/08/13 20:04:56 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 9 (MapPartitionsRDD[34] at take at cmd8.sc:1) (first 15 tasks are for partitions Vector(0))
21/08/13 20:04:56 INFO TaskSchedulerImpl: Adding task set 9.0 with 1 tasks resource profile 0
21/08/13 20:04:56 INFO TaskSetManager: Starting task 0.0 in stage 9.0 (TID 20) (192.168.0.68, executor driver, partition 0, NODE_LOCAL, 4453 bytes) taskResourceAssignments Map()
21/08/13 20:04:56 INFO Executor: Running task 0.0 in stage 9.0 (TID

res8: Array[Row] = Array([370002])

In [10]:
import org.apache.spark.sql.functions.max
flightData2015.select(max("count")).take(1)

21/08/13 20:05:07 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 20:05:07 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 20:05:07 INFO FileSourceStrategy: Output Data Schema: struct<count: int>
21/08/13 20:05:07 INFO MemoryStore: Block broadcast_14 stored as values in memory (estimated size 391.2 KiB, free 2002.4 MiB)
21/08/13 20:05:07 INFO MemoryStore: Block broadcast_14_piece0 stored as bytes in memory (estimated size 37.8 KiB, free 2002.3 MiB)
21/08/13 20:05:07 INFO BlockManagerInfo: Added broadcast_14_piece0 in memory on 192.168.0.68:57385 (size: 37.8 KiB, free: 2004.4 MiB)
21/08/13 20:05:07 INFO SparkContext: Created broadcast 14 from take at cmd9.sc:2
21/08/13 20:05:07 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
21/08/13 20:05:07 INFO SparkContext: Starting job: take at cmd9.sc:2
21/08/13 20:05:07 INFO DAGScheduler: Registering RDD 38 (take at cmd9.sc:2) as input to shuffle 2
21/08

take at cmd9.sc:2

1 / 1

21/08/13 20:05:07 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 15.8 KiB, free 2002.3 MiB)
21/08/13 20:05:07 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 8.0 KiB, free 2002.3 MiB)
21/08/13 20:05:07 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on 192.168.0.68:57385 (size: 8.0 KiB, free: 2004.4 MiB)
21/08/13 20:05:08 INFO SparkContext: Created broadcast 15 from broadcast at DAGScheduler.scala:1383
21/08/13 20:05:08 INFO DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 10 (MapPartitionsRDD[38] at take at cmd9.sc:2) (first 15 tasks are for partitions Vector(0))
21/08/13 20:05:08 INFO TaskSchedulerImpl: Adding task set 10.0 with 1 tasks resource profile 0
21/08/13 20:05:08 INFO TaskSetManager: Starting task 0.0 in stage 10.0 (TID 21) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4908 bytes) taskResourceAssignments Map()
21/08/13 20:05:08 INFO Executor: Running task 0.0 in stag

take at cmd9.sc:2

1 / 1

21/08/13 20:05:08 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 10.6 KiB, free 2002.3 MiB)
21/08/13 20:05:08 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 5.3 KiB, free 2002.3 MiB)
21/08/13 20:05:08 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on 192.168.0.68:57385 (size: 5.3 KiB, free: 2004.4 MiB)
21/08/13 20:05:08 INFO SparkContext: Created broadcast 16 from broadcast at DAGScheduler.scala:1383
21/08/13 20:05:08 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 11 (MapPartitionsRDD[41] at take at cmd9.sc:2) (first 15 tasks are for partitions Vector(0))
21/08/13 20:05:08 INFO TaskSchedulerImpl: Adding task set 11.0 with 1 tasks resource profile 0
21/08/13 20:05:08 INFO TaskSetManager: Starting task 0.0 in stage 11.0 (TID 22) (192.168.0.68, executor driver, partition 0, NODE_LOCAL, 4453 bytes) taskResourceAssignments Map()
21/08/13 20:05:08 INFO Executor: Running task 0.0 in stage 11.0 

import org.apache.spark.sql.functions.max

res9_1: Array[Row] = Array([370002])

In [14]:
val maxSql = spark.sql("""
  SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
  FROM flight_data_2015
  GROUP BY DEST_COUNTRY_NAME
  ORDER BY sum(count) DESC
  LIMIT 5
  """)
  maxSql.show()

21/08/13 19:54:49 INFO FileSourceStrategy: Pushed Filters: 
21/08/13 19:54:49 INFO FileSourceStrategy: Post-Scan Filters: 
21/08/13 19:54:49 INFO FileSourceStrategy: Output Data Schema: struct<DEST_COUNTRY_NAME: string, count: int>
21/08/13 19:54:49 INFO CodeGenerator: Code generated in 9.391935 ms
21/08/13 19:54:49 INFO CodeGenerator: Code generated in 10.94733 ms
21/08/13 19:54:49 INFO CodeGenerator: Code generated in 27.396062 ms
21/08/13 19:54:49 INFO CodeGenerator: Code generated in 48.322532 ms
21/08/13 19:54:49 INFO MemoryStore: Block broadcast_17 stored as values in memory (estimated size 391.2 KiB, free 2003.3 MiB)
21/08/13 19:54:49 INFO MemoryStore: Block broadcast_17_piece0 stored as bytes in memory (estimated size 37.9 KiB, free 2003.2 MiB)
21/08/13 19:54:49 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on 192.168.0.68:57333 (size: 37.9 KiB, free: 2004.5 MiB)
21/08/13 19:54:49 INFO SparkContext: Created broadcast 17 from show at cmd13.sc:8
21/08/13 19:54:49 INF

show at cmd13.sc:8

1 / 1

21/08/13 19:54:49 INFO MemoryStore: Block broadcast_18 stored as values in memory (estimated size 29.7 KiB, free 2003.2 MiB)
21/08/13 19:54:49 INFO MemoryStore: Block broadcast_18_piece0 stored as bytes in memory (estimated size 14.0 KiB, free 2003.2 MiB)
21/08/13 19:54:49 INFO BlockManagerInfo: Added broadcast_18_piece0 in memory on 192.168.0.68:57333 (size: 14.0 KiB, free: 2004.4 MiB)
21/08/13 19:54:49 INFO SparkContext: Created broadcast 18 from broadcast at DAGScheduler.scala:1383
21/08/13 19:54:49 INFO DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 12 (MapPartitionsRDD[45] at show at cmd13.sc:8) (first 15 tasks are for partitions Vector(0))
21/08/13 19:54:49 INFO TaskSchedulerImpl: Adding task set 12.0 with 1 tasks resource profile 0
21/08/13 19:54:49 INFO TaskSetManager: Starting task 0.0 in stage 12.0 (TID 23) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4908 bytes) taskResourceAssignments Map()
21/08/13 19:54:49 INFO Executor: Running task 0.0 in s

show at cmd13.sc:8 
 (kill)

0 / 200

21/08/13 19:54:49 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 34.8 KiB, free 2003.2 MiB)
21/08/13 19:54:49 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 17.3 KiB, free 2003.2 MiB)
21/08/13 19:54:49 INFO BlockManagerInfo: Added broadcast_19_piece0 in memory on 192.168.0.68:57333 (size: 17.3 KiB, free: 2004.4 MiB)
21/08/13 19:54:49 INFO SparkContext: Created broadcast 19 from broadcast at DAGScheduler.scala:1383
21/08/13 19:54:49 INFO DAGScheduler: Submitting 200 missing tasks from ShuffleMapStage 13 (MapPartitionsRDD[50] at show at cmd13.sc:8) (first 15 tasks are for partitions Vector(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14))
21/08/13 19:54:49 INFO TaskSchedulerImpl: Adding task set 13.0 with 200 tasks resource profile 0
21/08/13 19:54:49 INFO TaskSetManager: Starting task 2.0 in stage 13.0 (TID 24) (192.168.0.68, executor driver, partition 2, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
21/08/

In [ ]:
AmmoniteSparkSession.sync()

21/08/13 19:06:30 INFO SparkContext: Added JAR file:/Users/mehdi/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.13/scala-library-2.12.13.jar at spark://192.168.0.68:57008/jars/scala-library-2.12.13.jar with timestamp 1628906790743


res16: SparkSession = org.apache.spark.sql.SparkSession@13d78061

In [12]:
val maxSql = spark.sql("""
  SELECT DEST_COUNTRY_NAME, sum(count) 
  FROM flight_data_2015
  WHERE DEST_COUNTRY_NAME IN("FRANCE",
  				          -- "ITALY",
  							 "UNITED STATES"
							)
  GROUP BY DEST_COUNTRY_NAME
  LIMIT 5
  """)
 maxSql.show()

21/08/13 20:07:47 INFO FileSourceStrategy: Pushed Filters: In(DEST_COUNTRY_NAME, [FRANCE,UNITED STATES])
21/08/13 20:07:47 INFO FileSourceStrategy: Post-Scan Filters: DEST_COUNTRY_NAME#36 IN (FRANCE,UNITED STATES)
21/08/13 20:07:47 INFO FileSourceStrategy: Output Data Schema: struct<DEST_COUNTRY_NAME: string, count: int>
21/08/13 20:07:47 INFO CodeGenerator: Code generated in 9.17685 ms
21/08/13 20:07:47 INFO CodeGenerator: Code generated in 16.997501 ms
21/08/13 20:07:47 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 391.2 KiB, free 2004.2 MiB)
21/08/13 20:07:47 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 37.8 KiB, free 2004.2 MiB)
21/08/13 20:07:47 INFO BlockManagerInfo: Added broadcast_21_piece0 in memory on 192.168.0.68:57385 (size: 37.8 KiB, free: 2004.6 MiB)
21/08/13 20:07:47 INFO SparkContext: Created broadcast 21 from show at cmd11.sc:11
21/08/13 20:07:47 INFO FileSourceScanExec: Planning scan with bin p

show at cmd11.sc:11 
 (kill)

0 / 1

21/08/13 20:07:47 INFO MemoryStore: Block broadcast_22 stored as values in memory (estimated size 31.7 KiB, free 2004.1 MiB)
21/08/13 20:07:47 INFO MemoryStore: Block broadcast_22_piece0 stored as bytes in memory (estimated size 14.6 KiB, free 2004.1 MiB)
21/08/13 20:07:47 INFO BlockManagerInfo: Added broadcast_22_piece0 in memory on 192.168.0.68:57385 (size: 14.6 KiB, free: 2004.5 MiB)
21/08/13 20:07:47 INFO SparkContext: Created broadcast 22 from broadcast at DAGScheduler.scala:1383
21/08/13 20:07:47 INFO DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 15 (MapPartitionsRDD[55] at show at cmd11.sc:11) (first 15 tasks are for partitions Vector(0))
21/08/13 20:07:47 INFO TaskSchedulerImpl: Adding task set 15.0 with 1 tasks resource profile 0
21/08/13 20:07:47 INFO TaskSetManager: Starting task 0.0 in stage 15.0 (TID 225) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4908 bytes) taskResourceAssignments Map()
21/08/13 20:07:47 INFO Executor: Running task 0.0 in

show at cmd11.sc:11 
 (kill)

0 / 200

21/08/13 20:07:47 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 33.1 KiB, free 2004.1 MiB)
21/08/13 20:07:47 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory (estimated size 16.7 KiB, free 2004.1 MiB)
21/08/13 20:07:47 INFO BlockManagerInfo: Added broadcast_23_piece0 in memory on 192.168.0.68:57385 (size: 16.7 KiB, free: 2004.5 MiB)
21/08/13 20:07:47 INFO SparkContext: Created broadcast 23 from broadcast at DAGScheduler.scala:1383
21/08/13 20:07:47 INFO DAGScheduler: Submitting 200 missing tasks from ShuffleMapStage 16 (MapPartitionsRDD[58] at show at cmd11.sc:11) (first 15 tasks are for partitions Vector(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14))
21/08/13 20:07:47 INFO TaskSchedulerImpl: Adding task set 16.0 with 200 tasks resource profile 0
21/08/13 20:07:47 INFO TaskSetManager: Starting task 0.0 in stage 16.0 (TID 226) (192.168.0.68, executor driver, partition 0, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
2